## Check data and build input reading pipeline

In [1]:
from __future__ import print_function

import os
import sys

import numpy as np
import pandas as pd
import time
import glob
import re
import random
import datetime

from sklearn.model_selection import train_test_split
from PIL import Image
import cv2

import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--gpu_id', type = str)
parser.add_argument('--split_id', type = str, help = 'split_id, from 0 to 7')
parser.add_argument('--full_random', default = 0)
parser.add_argument('--with_transfer', default = 1)
parser.add_argument('--message', type = str, help = "Recording experiment infomation")
FLAGS = parser.parse_args([])
print(FLAGS)

FLAGS.gpu_id = "1"
FLAGS.full_random = 1
FLAGS.with_transfer = 0
FLAGS.message = "run it"

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = str(FLAGS.gpu_id)
import tensorflow as tf

#from config import Basic_Setup, Hparams, Augmentation_Setup
from DataGenerator import Create_Data_Generator

sys.path.append("/home/seanyu/Common_tools/")
from model import create_model
from trainer import model_trainer
from CONFIG_MODEL import PRETRAIN_MODEL_SETUP
from Recording import Experiment_Recoding
from TF_callbacks import (
    EarlyStopping, 
    Model_checkpoint, 
    ReduceLROnPlateau, 
    Model_Timer,
    Run_collected_functions
)

/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  retu

Namespace(full_random=0, gpu_id=None, message=None, split_id=None, with_transfer=1)


/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
class Basic_Setup(object):
    data_dir = "/data/seanyu/UPMC_pathology/Pathology_Analysis/data/" # path to data
    label_dir = "/data/seanyu/UPMC_pathology/Pathology_Analysis/splits/" # path to label
    result_dir = "/data/seanyu/UPMC_pathology/result/resnet50_RandomSplit_FromScratch" # path to store results
    model_saving_name = "model" # model name prefix
    do_augmentation = True
    
    n_threads = 8
    dq_size = 50
    
# ======================= #
try:
    import imgaug as ia
    from imgaug import augmenters as iaa
except:
    print("Import Error, Please make sure you have imgaug")
        
try:
    sys.path.append("Common_tools/")
    from customized_imgaug_func import keypoint_func, img_channelswap
except:
    print("Warning, if you used customized imgaug function")

class Augmentation_Setup(object):  
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)
    lesstimes = lambda aug: iaa.Sometimes(0.2, aug)
    
    augmentation = iaa.Sequential([
        iaa.Fliplr(0.5, name="FlipLR"),
        iaa.Flipud(0.5, name="FlipUD"),
        iaa.ContrastNormalization((0.8, 1.2), name = "Contrast"),
        iaa.Add((-15, 15), per_channel = 0.5),
   #     iaa.OneOf([iaa.Multiply((0.8, 1.2), per_channel = 0.5, name = "Multiply"),
   #                iaa.AddToHueAndSaturation((-15,15),name = "Hue"),
   #               ]),
        sometimes(iaa.GaussianBlur((0, 1.5), name="GaussianBlur")),
        iaa.OneOf([iaa.Affine(rotate = 90),
                   iaa.Affine(rotate = 180),
                   iaa.Affine(rotate = 270)]),
        sometimes(iaa.Affine(
                    scale = (0.6,1.2),
                    #translate_percent = (-0.2, 0.2),
                    rotate = (-15, 15),
                    mode = 'wrap'
                    ))
    ])
    
# ============================== #
import tensorflow as tf
class Hparams(object):
    """
    Model related setup
    """
    input_size = (256, 256, 3)
    batch_size = 32
    epochs = 200
    n_batch = 300
    n_class = 2
    class_weight = None # leave None if don't use it
    optimizer = 'adam'
    pretrain_model = "resnet_50"
    
    learning_rate = 0.00017
    reduce_lr_options = {'lr':learning_rate, 'factor':0.5, 'patience':4}
    earlystop_options = {'min_delta': 1e-4, 'patience':9}

/home/seanyu/.conda/envs/tf18_keras/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
for i in range(5):
    tf.reset_default_graph()
    FLAGS.split_id = str(i)
    recording_manager = Experiment_Recoding(Basic_Setup.result_dir, message=FLAGS.message)
    start_time = time.time()
    df = pd.DataFrame()
    for root_path, middle_path, target_file in os.walk(Basic_Setup.label_dir):
        if len(target_file) == 0:
            pass
        else:
            for sub_file in target_file:
                full_name = os.path.join(root_path, sub_file)
                tmp_files = pd.read_csv(full_name, header=None)
                df = pd.concat((df, tmp_files), axis = 0)
    df.columns = ["pid","cate"]
    df.drop_duplicates(inplace=True)
    df['img_path'] = df['pid'].apply(lambda x: os.path.join(Basic_Setup.data_dir, 'image' + str (x) + ".png"))
    training_files, validation_files = train_test_split(df, test_size = 0.1, stratify =df.cate.values.astype('int')) # around 1:10

    train_0 = training_files[training_files['cate'] == 0]
    train_1 = training_files[training_files['cate'] == 1]
    train_0.reset_index(drop=True, inplace=True)
    train_1.reset_index(drop=True, inplace=True)

    data_gen = Create_Data_Generator(df = [train_0, train_1],
                                     image_size=Hparams.input_size,
                                     batch_size=Hparams.batch_size, n_classes=Hparams.n_class,
                                     nd_inputs_preprocessing_handler=PRETRAIN_MODEL_SETUP.CORRESPONDING_PREPROC[Hparams.pretrain_model],
                                     do_augment=Basic_Setup.do_augmentation, aug_params=Augmentation_Setup.augmentation
                                    )

    data_gen.start_train_threads(jobs=Basic_Setup.n_threads, dq_size=Basic_Setup.dq_size)
    train_gen = data_gen.get_data()
    x_val, y_val = data_gen.get_evaluate_data(validation_files)
    print(x_val.shape)

    cb_dict = {
        'reduce_lr' : ReduceLROnPlateau(lr=Hparams.reduce_lr_options['lr'], 
                                        factor=Hparams.reduce_lr_options['factor'], 
                                        patience=Hparams.reduce_lr_options['patience']),
        'earlystop' : EarlyStopping(min_delta = Hparams.earlystop_options['min_delta'], 
                                    patience= Hparams.earlystop_options['patience']),
        'checkpoint' : Model_checkpoint(model_name=Basic_Setup.result_dir + '/' +  Basic_Setup.model_saving_name, 
                                        save_best_only=True),
        'model_timer': Model_Timer(),
    }

    callback_dict = {
        'on_session_begin':[cb_dict['model_timer']], # start of a session
        'on_batch_begin':[], # start of a training batch
        'on_batch_end':[], # end of a training batch
        'on_epoch_begin':[], # start of a epoch
        'on_epoch_end':[cb_dict['earlystop'], 
                        cb_dict['reduce_lr'],
                        cb_dict['checkpoint']], # end of a epoch
        'on_session_end':[cb_dict['model_timer']] # end of a session
        }
    callback_manager = Run_collected_functions(callback_dict)

    model_ops, metric_history = create_model(Hparams)
    trainer = model_trainer(hflags=Hparams, # hyper-parameters
                            data_gen=data_gen, # data generator, for get infos
                            data_gen_get_data = train_gen, # for yield patches
                            model_ops=model_ops, # model graph
                            metric_history=metric_history, # metric recording
                            callback_manager=callback_manager # runable callbacks
                            )
    trainer.initalize()
    if FLAGS.with_transfer:
        pretrain_ckpt = PRETRAIN_MODEL_SETUP.PRETRAIN_DICT[Hparams.pretrain_model]
        trainer.restore(model_to_restore=pretrain_ckpt, partial_restore=True)
    trainer.do_training(cb_dict=cb_dict, validation_set=(x_val, y_val))

    val_pred = trainer.predict(x_= x_val, 
                               model_to_restore=os.path.join(Basic_Setup.result_dir, Basic_Setup.model_saving_name) + '.ckpt', 
                               bz=Hparams.batch_size)

    val_pred = np.concatenate([item for sublist in val_pred for item in sublist])
    acc = np.sum(val_pred.argmax(axis = 1) == y_val.argmax(axis = 1)) / len(y_val)
    print(acc)

    validation_files['y_pred'] = val_pred[:,1]

    if not FLAGS.full_random:
        output_df_name = os.path.join(Basic_Setup.result_dir, 'result_sp' + str(FLAGS.split_id) + '.csv')
        history_file_name = os.path.join(Basic_Setup.result_dir, 'log_sp' + str(FLAGS.split_id) + '.csv')
    else:
        output_df_name = os.path.join(Basic_Setup.result_dir, 'result_randomsp' + str(FLAGS.split_id) + '.csv')
        history_file_name = os.path.join(Basic_Setup.result_dir, 'log_randomsp' + str(FLAGS.split_id) + '.csv')

    validation_files.to_csv(path_or_buf=output_df_name, index=False)

    #recording_manager.write_new_msg('Validation Performance Accuracy: %.3f' % (acc) )

    history_logs = pd.DataFrame({'train_accuracy': trainer.metric_history['accuracy']['train'],
                                 'valid_accuracy': trainer.metric_history['accuracy']['valid']})

    history_logs.to_csv(history_file_name, index=False)

/data/seanyu/UPMC_pathology/result/resnet50_RandomSplit_FromScratch has already exist, overwrite it
Write recording file to /data/seanyu/UPMC_pathology/result/resnet50_RandomSplit_FromScratch/msg.txt
copying /home/seanyu/project/UPMC_mitosis/config.py to /data/seanyu/UPMC_pathology/result/resnet50_RandomSplit_FromScratch/config.py
copying /home/seanyu/project/UPMC_mitosis/model.py to /data/seanyu/UPMC_pathology/result/resnet50_RandomSplit_FromScratch/model.py
(1095, 256, 256, 3)
== INITIALIZE PARAMETERS ==


Training batch:   0%|          | 0/300 [00:00<?, ?batch/s]

Start time: Tue Oct  2 12:16:46 2018


Batch: 300, Training loss/acc: 0.46/0.78: 100%|██████████| 300/300 [01:23<00:00,  3.61batch/s]


Epoch: 1/200
train loss: 0.46340322494506836 | val loss: 0.25835558772087097
== Validation loss has an improvement, save model ==


...

Epoch: 28/200
train loss: 0.0669437050819397 | val loss: 0.10697543621063232


Batch: 300, Training loss/acc: 0.06/0.98: 100%|██████████| 300/300 [01:12<00:00,  4.14batch/s]


Epoch: 29/200
train loss: 0.06184804439544678 | val loss: 0.11308847367763519
Earlystop criteria met
Stopping Thread 0
Stopping Thread 1
Stopping Thread 2
Stopping Thread 3
Stopping Thread 4
Stopping Thread 5
Stopping Thread 6
Stopping Thread 7
Stopping Thread 8
End time: Tue Oct  2 15:31:25 2018
Total elapse time: 00:37:07
Stopping Thread 0
Stopping Thread 1
Stopping Thread 2
Stopping Thread 3
Stopping Thread 4
Stopping Thread 5
Stopping Thread 6
Stopping Thread 7
Stopping Thread 8
End time: Tue Oct  2 15:31:34 2018
Total elapse time: 00:37:16
INFO:tensorflow:Restoring parameters from /data/seanyu/UPMC_pathology/result/resnet50_RandomSplit_FromScratch/model.ckpt


100%|██████████| 35/35 [00:03<00:00,  9.92it/s]


0.9662100456621004
